In [1]:
import sys
sys.path.insert(0, '../Scripts')
from qr_query import qr_search
qr = input('Introduce un código QR: ')
qr_search(qr)

Introduce un código QR: 601170500100130
Connection to local Mongo successful.
Connection to Atlas successful.
No images were found locally, searching in the cloud now.
No images were found in Atlas.
No data was found locally, searching in the cloud now.


,No. Imagen,Validez,Fecha + 5hrs,Ciudad,ESAT6,CFP10,RV1681,P24,P26,Control
4,0,Valid,2017-05-19 15:06,Zapopan,Positive,Positive,Negative,Positive,Negative,Negative
1,1,Valid,2017-08-30 20:28,Zapopan,Positive,Positive,Negative,Positive,Negative,Negative
0,2,Valid,2017-09-04 17:44,Zapopan,Positive,Positive,Negative,Positive,Negative,Negative
2,3,Valid,2017-10-19 16:10,Zapopan,Positive,Positive,Positive,Positive,Negative,Negative
3,4,Valid,2017-12-21 21:57,Zapopan,Positive,Positive,Negative,Positive,Negative,Negative
